# Импорт датасета

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

filepaths = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepaths.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(filepaths[0])
df.head()

## Выберем только тексты и рубрики Экономика и Спорт

In [ ]:
news = df[['text', 'rubric']]
news = news[news['rubric'].isin(['Экономика', 'Спорт'])]
news = news.reset_index(drop=True)
news

# Используем UDP для предобработки

In [ ]:
!pip install wget
!pip install ufal.udpipe

In [ ]:
import wget
from ufal.udpipe import Model, Pipeline
import os, re, sys

udp_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udp_model_filename = udp_model_url.split('/')[-1]

if not os.path.isfile(udp_model_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    udp_model_filename = wget.download(udp_model_url)

print('\nLoading the model...', file=sys.stderr)
udp_model = Model.load(udp_model_filename)
udp_process_pipeline = Pipeline(udp_model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


def tag(sentance='Текст нужно передать функции в виде предложения!'):
    modelfile='udpipe_syntagrus.model'
    processed = udp_process_pipeline.process(sentance)
    
    result = []
    processed = processed.split('\n')
    for line in processed:
        if line.startswith('#') or not line:
            continue
        line = line.split('\t')
        if line[3] not in ['PUNCT', 'X']:
            result.append(line[2] + '_' + line[3])
    
    return result

In [ ]:
#Проверим работу размечающей функции
print(tag("Съешь ещё этих мягких французских булок, да выпей же чаю."))

In [ ]:
news['text'] = news['text'].apply(tag)
news['rubric'] = news['rubric'].apply(lambda x: 0 if x == 'Экономика' else 1)
print(news)

# Применим модель от RusVectores для векторизации

In [ ]:
!pip install gensim

In [ ]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [ ]:
#Перведём каждое слово в предложении в вектор, затем по посчитаем среднее
def sent2vec(sent):
    sent = list(filter(lambda x: x in model.vocab, sent))
    vects = [model.wv[word] for word in sent]
    return np.mean(vects, axis=0)
    
news['text'] = news['text'].apply(sent2vec)
print(news)

In [ ]:
features = pd.DataFrame(news['text'][0].reshape(-1, len(news['text'][0])))
for i in range(1, news['text'].shape[0]):
    features = features.append(pd.DataFrame(news['text'][i].reshape(-1, len(news['text'][i]))))
features = features.reset_index(drop=True)

targets = pd.Series(news['rubric'][0])
for i in range(1, news['rubric'].shape[0]):
    targets = targets.append(pd.Series(news['rubric'][i]))
targets = targets.reset_index(drop=True)

# Визуализируем

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from sklearn import manifold, datasets

In [ ]:
vecs_embedded = manifold.TSNE(n_components=2).fit_transform(features)
clrs = np.asarray(list(targets.apply(lambda x: [1., 0., 0.] if x == 1 else [0., 1., 0.]).to_numpy()))

## Зелёные - экономика, красные - спорт

In [ ]:
plt.scatter(vecs_embedded.T[0], vecs_embedded.T[1], c=clrs)
plt.show()